This is an code of the following paper that can be verified with `Google Colab`.

[Learning to Simulate Complex Physics with Graph Networks](https://deepmind.com/research/open-source/Learning-to-Simulate-Complex-Physics-with-Graph-Networks)

Specifically, some part of the code of the following repo (visualization of learning results) has been changed.

https://github.com/deepmind/deepmind-research/tree/master/learning_to_simulate

Consider using `Colab Pro` or `Colab Pro+`

```
@inproceedings{sanchezgonzalez2020learning,
  title={Learning to Simulate Complex Physics with Graph Networks},
  author={Alvaro Sanchez-Gonzalez and
          Jonathan Godwin and
          Tobias Pfaff and
          Rex Ying and
          Jure Leskovec and
          Peter W. Battaglia},
  booktitle={International Conference on Machine Learning},
  year={2020}
}
```

In [ ]:
# T4, P100 or better required
!nvidia-smi

In [ ]:
# Download Repo
!git clone https://github.com/deepmind/deepmind-research.git

In [ ]:
%cd /content/deepmind-research

In [ ]:
# Install Dependencies
!pip install -r learning_to_simulate/requirements.txt
!mkdir -p /tmp/rollous

**Restart the runtime**

In [ ]:
%cd /content/deepmind-research

In [ ]:
# Download the dataset
!mkdir -p /tmp/datasets
!bash ./learning_to_simulate/download_dataset.sh WaterDrop /tmp/datasets

In [ ]:
# visualize model training results
%load_ext tensorboard

In [ ]:
# Prepare to visualize model training results
%tensorboard --logdir="/content/drive/MyDrive/Colab Notebooks/models/WaterDrop" # here is your convenient dir

In [ ]:
# Learning model
!mkdir -p /tmp/models
!python -m learning_to_simulate.train \
    --data_path=/tmp/datasets/WaterDrop \
    --model_path="/content/drive/MyDrive/Colab Notebooks/models/WaterDrop"

In [ ]:
# Generate several trajectory rollouts in the test set
# Can be manually stopped once sufficient rollout has been generated
!mkdir -p /tmp/rollouts
!python -m learning_to_simulate.train \
    --mode="eval_rollout" \
    --data_path=/tmp/datasets/WaterDrop \
    --model_path="/content/drive/MyDrive/Colab Notebooks/models/WaterDrop" \
    --output_path=/tmp/rollouts/WaterDrop

In [ ]:
import glob
import pickle
import time

from absl import app
from absl import flags

from matplotlib import animation
from matplotlib import rc
import matplotlib.pyplot as plt
import numpy as np

rollout_path = '/tmp/rollouts/WaterDrop/rollout_test_0.pkl' # Specify rollout num

step_stride = 3
block_on_show = True

TYPE_TO_COLOR = {
    3: "black",  # Boundary particles.
    0: "green",  # Rigid solids.
    7: "magenta",  # Goop.
    6: "gold",  # Sand.
    5: "blue",  # Water.
}

with open(rollout_path, "rb") as file:
    rollout_data = pickle.load(file)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
plot_info = []
for ax_i, (label, rollout_field) in enumerate(
    [("Ground truth", "ground_truth_rollout"),
        ("Prediction", "predicted_rollout")]):
    # Append the initial positions to get the full trajectory.
    trajectory = np.concatenate([
        rollout_data["initial_positions"],
        rollout_data[rollout_field]], axis=0)
    ax = axes[ax_i]
    ax.set_title(label)
    bounds = rollout_data["metadata"]["bounds"]
    ax.set_xlim(bounds[0][0], bounds[0][1])
    ax.set_ylim(bounds[1][0], bounds[1][1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1.)
    points = {
        particle_type: ax.plot([], [], "o", ms=2, color=color)[0]
        for particle_type, color in TYPE_TO_COLOR.items()}
    plot_info.append((ax, trajectory, points))

num_steps = trajectory.shape[0]

def update(step_i):
    outputs = []
    for _, trajectory, points in plot_info:
        for particle_type, line in points.items():
            mask = rollout_data["particle_types"] == particle_type
            line.set_data(trajectory[step_i, mask, 0],
                            trajectory[step_i, mask, 1])
            outputs.append(line)
    return outputs

unused_animation = animation.FuncAnimation(
    fig, update,
    frames=np.arange(0, num_steps, step_stride), interval=10)
rc('animation', html='jshtml')
unused_animation